In [ ]:
!pip -q install onnxruntime

In [ ]:
token = "insert-user-token-here"

In [60]:
model_name = "jukebox"

In [32]:
import pandas as pd
import pickle
import json
import onnxruntime as rt
import numpy as np
import onnx

import requests
from urllib.parse import urljoin

In [ ]:
X_train = pd.read_parquet('../2-dev_datascience/models/jukebox/1/artifacts/X_train.parquet')[:5000]

In [ ]:
with open('../2-dev_datascience/models/jukebox/1/artifacts/scaler.pkl', 'rb') as handle:
    scaler = pickle.load(handle)

In [ ]:
data = scaler.transform(X_train.values).tolist()

In [ ]:
sess = rt.InferenceSession("../2-dev_datascience/models/jukebox/1/model.onnx", providers=rt.get_available_providers())
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
y_pred_temp = sess.run([output_name], {input_name: data.astype(np.float32)})

In [16]:
training_data = {
    "model_name": model_name,
    "data_tag": "TRAINING",
    "request": {
        "inputs": [
            {
                "name": input_name,
                "shape": np.shape(data),
                "datatype": "FP32",
                "data": data
            }
        ]    
    },
    "response": {
        "model_name": model_name,
        "model_version": "1",
        "outputs": [
            {
                "name": output_name,
                "datatype": "FP32",
                "shape": np.shape(y_pred_temp[0]),
                "data": y_pred_temp[0].tolist()
            }
        ]
    }
}


In [41]:
base_url = "your-trustyai-service"
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

In [ ]:
# Upload data
endpoint = "data/upload"
url = urljoin(base_url, endpoint)
response = requests.post(url, headers=headers, json=training_data)
print(response.text)

In [ ]:
# Change the input and output names
endpoint = "info/names"
url = urljoin(base_url, endpoint)

payload = {
    "modelId": model_name,
    "inputMapping": {
        "dense_input-0": "is_explicit",
        "dense_input-1": "duration_ms",
        "dense_input-2": "danceability",
        "dense_input-3": "energy",
        "dense_input-4": "key",
        "dense_input-5": "loudness",
        "dense_input-6": "mode",
        "dense_input-7": "speechiness",
        "dense_input-8": "acousticness",
        "dense_input-9": "instrumentalness",
        "dense_input-10": "liveness",
        "dense_input-11": "valence",
        "dense_input-12": "tempo"
    },
    "outputMapping": {
    }
}

response = requests.post(url, headers=headers, json=payload)
print(response.text)

In [ ]:
# Monitor meanshift
# A p-value of 1.0 indicates a very high likelihood that the test and training data come from the same distribution, while a p-value < 0.05 indicates a statistically significant drift between train and test set
endpoint = "/metrics/drift/meanshift/request"
url = urljoin(base_url, endpoint)

payload = {
    "modelId": model_name,
    "referenceTag": "TRAINING"
}

response = requests.post(url, headers=headers, json=payload)
print(response.text)

In [ ]:
# Get bias for a specific field-couple
endpoint = "/metrics/group/fairness/spd"
url = urljoin(base_url, endpoint)

payload = {
    "modelId": model_name,
    "protectedAttribute": "is_explicit",
    "privilegedAttribute": 1.0,
    "unprivilegedAttribute": 0.0,
    "outcomeName": "dense_4-13",
    "favorableOutcome": 0.5,
    "batchSize": 1000
}

response = requests.post(url, headers=headers, json=payload)
print(response.text)

In [ ]:
# Monitor the average value
endpoint = "/metrics/identity/request"
url = urljoin(base_url, endpoint)

payload = {
    "modelId": model_name,
    "columnName": "duration_ms",
    "batchSize": 256,
}

response = requests.post(url, headers=headers, json=payload)
print(response.text)

In [ ]:
# See what we have registered
endpoint = "/info"
url = urljoin(base_url, endpoint)
response = requests.get(url, headers=headers)
print(response.text)

### Then we need to send one request to start the tracking

In [ ]:
deployed_model_name = "jukebox"
infer_endpoint = "https://jukebox-robert-test.apps.cluster-5vbcm.5vbcm.sandbox692.opentlc.com"
infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"

def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "dense_input",
                "shape": [1, 13],
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    response = requests.post(infer_url, json=json_data, verify=False)
    response_dict = response.json()
    return response_dict['outputs'][0]['data']

prediction = rest_request(data[0])